In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


# Import dataset from npz file
with np.load('/Users/jvelasquez/Virginia_Tech/Spring_2023/ECE_4806/pedestal_svm/pedestal_color_dataset_v2.npz') as data:
    print(data.files)
    images = data['images']
    labels = data['labels']

['images', 'labels']


In [2]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

# Print the smallest and largest values in a image
print("Smallest value in a image: ", np.min(X_train[0]))
print("Largest value in a image: ", np.max(X_train[1]))

# Preprocess the images
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Flatten the images
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Create the SVM classifier
clf = SVC(kernel='rbf', C=10, gamma='scale', class_weight=None)
clf.fit(X_train, y_train)

# Evaluate the classifier
print("Accuracy: ", clf.score(X_test, y_test))


Smallest value in a image:  0
Largest value in a image:  253
Accuracy:  0.9961240310077519


In [20]:
# Print the size, in bytes, of the clf object
print("Size of the classifier object: ", clf.__sizeof__())

Size of the classifier object:  32


In [3]:
# Use GridSearchCV to find the best parameters for the SVM classifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    "kernel": ["linear", "poly", "rbf"],
    "C": [0.1, 1, 10],
    "gamma": ["scale", "auto"],
    "class_weight": [None, "balanced"]
}

# Create an SVM classifier object
svm = SVC()

# Use GridSearchCV to find the best combination of hyperparameters, running verbose
grid_search = GridSearchCV(svm, param_grid, cv=5, n_jobs=2, verbose=4, return_train_score=True, pre_dispatch=4)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print("Best parameters:", grid_search.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 2/5] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=(train=1.000, test=0.818) total time= 1.6min
[CV 1/5] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=(train=1.000, test=0.818) total time= 1.6min
[CV 3/5] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=(train=1.000, test=0.853) total time= 1.5min


/Users/jvelasquez/miniforge3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 4/5] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=(train=1.000, test=0.873) total time= 1.5min
[CV 5/5] END C=0.1, class_weight=None, gamma=scale, kernel=linear;, score=(train=1.000, test=0.838) total time= 1.5min
[CV 1/5] END C=0.1, class_weight=None, gamma=scale, kernel=poly;, score=(train=1.000, test=0.846) total time= 1.6min
[CV 2/5] END C=0.1, class_weight=None, gamma=scale, kernel=poly;, score=(train=1.000, test=0.860) total time= 1.5min
[CV 3/5] END C=0.1, class_weight=None, gamma=scale, kernel=poly;, score=(train=1.000, test=0.902) total time= 1.5min
[CV 4/5] END C=0.1, class_weight=None, gamma=scale, kernel=poly;, score=(train=1.000, test=0.894) total time= 1.6min
[CV 5/5] END C=0.1, class_weight=None, gamma=scale, kernel=poly;, score=(train=1.000, test=0.852) total time= 1.5min
[CV 1/5] END C=0.1, class_weight=None, gamma=scale, kernel=rbf;, score=(train=0.677, test=0.671) total time= 2.4min
[CV 2/5] END C=0.1, class_weight=None, gamma=scale, kernel=rb

In [4]:
# Evaluate the classifier with the best hyperparameters
# Create the SVM classifier
clf = SVC(kernel='rbf', C=10, gamma='scale', class_weight=None)
clf.fit(X_train, y_train)

# Evaluate the classifier
print("Accuracy: ", clf.score(X_test, y_test))

Accuracy:  0.9720670391061452


In [6]:
# Load a test image with Pillow
from PIL import Image

image = Image.open('/Users/jvelasquez/Downloads/jpg/IMG_0810.jpg')

# Downscale the image, maintaining the aspect ratio
image.thumbnail((640, 640))

if image.size != (640, 480):
    # Rotate the image on its side
    image = image.rotate(90, expand=True)

# Convert image to a numpy array
image = np.array(image)
print(image.shape)


(480, 640, 3)


In [7]:
# Make a prediciton on the new image
image = image.astype('float32') / 255
image = image.reshape(1, -1)
print(image.shape)

prediction = clf.predict(image)
print(prediction)

if prediction == 0:
    print("Prediction: Red")
elif prediction == 1:
    print("Prediction: Green")
else:
    print("Prediction: White")

(1, 921600)
[1]
Prediction: Green


In [ ]:
# save the model to disk
import pickle

filename = 'pedestal_color_classifier.pkl'
pickle.dump(clf, open(filename, 'wb'))